In [1]:
!pip install SpeechRecognition
!apt-get install -y portaudio19-dev python3-pyaudio
!pip install pyaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 22.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev python3-pyaudio
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 213 kB of archives.
After this operation, 1,043 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubunt

In [3]:
!pip install --force-reinstall pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp312-cp312-linux_x86_64.whl size=68668 sha256=e31a3cb028bb71d63ee5787de068675ecd738a8c13824269217199e6cdce3297
  Stored in directory: /root/.cache/pip/wheels/68/c7/33/c6a6b210cb5819ec5c219928c794a447742a7d86d21c0b92e6
Successfully built pyaudio
  Attempting uninstall: pyaudio
    Found existing installation: PyAudio 0.2.11
error: uninstall-distutils-installed-package

× Cannot uninstall PyAudio 0.2.11
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [5]:
!pip install --upgrade pyaudio --ignore-installed

  Using cached pyaudio-0.2.14-cp312-cp312-linux_x86_64.whl


In [7]:
!pip install SpeechRecognition pydub

In [18]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [30]:
from IPython.display import Javascript, display
from base64 import b64decode
from google.colab import output
import io
import pydub

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
var record = async function record() {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  let chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await sleep(5000); // record for 5 seconds
  recorder.stop();
  await sleep(200); // wait for stop event
  const blob = new Blob(chunks);
  const dataUrl = await new Promise(resolve => {
    const reader = new FileReader();
    reader.onload = () => resolve(reader.result);
    reader.readAsDataURL(blob);
  });
  return dataUrl.split(',')[1];
}
"""

def record_audio_and_convert(filename="audio.wav", duration=5):
    """Records audio and converts it to a proper WAV file."""
    display(Javascript(RECORD))
    print(f"Starting recording for {duration} seconds...")
    js_command = f"record()"
    audio_base64 = output.eval_js(js_command)
    print("Recording finished. Converting to WAV...")

    # Decode base64 to bytes and convert with pydub
    audio_bytes = b64decode(audio_base64)
    audio_segment = pydub.AudioSegment.from_file(io.BytesIO(audio_bytes), format="webm")
    audio_segment.export(filename, format="wav")

    print(f"Audio saved to '{filename}'.")
    return filename

audio_file_path = record_audio_and_convert(duration=5)

<IPython.core.display.Javascript object>

Starting recording for 5 seconds...
Recording finished. Converting to WAV...
Audio saved to 'audio.wav'.


In [31]:
import speech_recognition as sr

def recognize_speech_from_file(audio_path):
    """Transcribes audio from a file."""

    r = sr.Recognizer()

    try:
        # Use the audio file as the source
        with sr.AudioFile(audio_path) as source:
            audio_data = r.record(source)

        # Recognize speech using Google's Web Speech API
        text = r.recognize_google(audio_data)
        print(f"You said: {text}")

    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
    except FileNotFoundError:
        print("Error: The audio file was not found. Please ensure the recording step ran successfully.")

# Use the file path from the previous step
recognize_speech_from_file(audio_file_path)

You said: 1 2 3 4 5
